In [ ]:
!pip install onnxruntime datasets transformers numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
!pip install onnxruntime

In [ ]:
import onnxruntime as ort
import numpy as np
import time
import tracemalloc
from datasets import load_dataset
from transformers import AutoTokenizer

# Step 1: Load dataset

In [ ]:
dataset = load_dataset("glue","sst2")

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
train_data = dataset["train"]
train_data

Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 67349
})

In [ ]:
sentence = [sample["sentence"] for sample in train_data.select(range(10))]

In [ ]:
sentence

['hide new secretions from the parental units ',
 'contains no wit , only labored gags ',
 'that loves its characters and communicates something rather beautiful about human nature ',
 'remains utterly satisfied to remain the same throughout ',
 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ',
 "that 's far too tragic to merit such superficial treatment ",
 'demonstrates that the director of such hollywood blockbusters as patriot games can still turn out a small , personal film with an emotional wallop . ',
 'of saucy ',
 "a depressed fifteen-year-old 's suicidal poetry ",
 "are more deeply thought through than in most ` right-thinking ' films "]

# Step 2: Tokenization

In [ ]:
tokenize = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized = tokenize(sentence,padding=True,truncation=True,return_tensors="np")

In [ ]:
tokenized

{'input_ids': array([[  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  101,  3397,  2053, 15966,  1010,  2069,  4450,  2098, 18201,
         2015,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  101,  2008,  7459,  2049,  3494,  1998, 10639,  2015,  2242,
         2738,  3376,  2055,  2529,  3267,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  101,  3464, 12580,  8510,  2000,  3961,  1996,  2168,  2802,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  101,  2006,  1996,  5409,  7195,

In [ ]:
input_ids = np.array(tokenized["input_ids"],dtype=np.int64)

In [ ]:
attention_mask = np.array(tokenized["attention_mask"],dtype=np.int64)

# Step 3: Download ONNX model

In [ ]:
onnx_model_path = "/content/distilbert-sst2.onnx"

In [ ]:
!wget -O {onnx_model_path} "https://huggingface.co/optimum/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/model.onnx"

--2025-03-19 13:17:51--  https://huggingface.co/optimum/distilbert-base-uncased-finetuned-sst-2-english/resolve/main/model.onnx
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/d0/6e/d06eae48d74816d2ffbbe257ebf858e495147e679482d296e25f4e8bc92ad50d/d59a5577be6a248acf73055c5c81a7a05fc436548f302c7847a6cd1164b0c216?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.onnx%3B+filename%3D%22model.onnx%22%3B&Expires=1742393871&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjM5Mzg3MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9kMC82ZS9kMDZlYWU0OGQ3NDgxNmQyZmZiYmUyNTdlYmY4NThlNDk1MTQ3ZTY3OTQ4MmQyOTZlMjVmNGU4YmM5MmFkNTBkL2Q1OWE1NTc3YmU2YTI0OGFjZjczMDU1YzVjODFhN2EwNWZjNDM2NTQ4ZjMwMmM3ODQ3YTZjZDExNjRiMGMyMTY%7EcmVzcG

#  Step 4:Load the ONNX model

In [ ]:
import onnxruntime as ort
session = ort.InferenceSession(onnx_model_path)

# Step 5: Measure Time & Memory Usage

In [ ]:
tracemalloc.start()
start_time = time.time()


# Run inference
output = session.run(None, {"input_ids": input_ids, "attention_mask": attention_mask})


end_time = time.time()
current,peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

# step 6: Results

In [ ]:
print("\n Inference Completed!")

print(f"Exection Time : {end_time - start_time:.4f} Seconds")
print(f"Memory usage : {peak/1024:.2f}Kb")


 Inference Completed!
Exection Time : 2.5495 Seconds
Memory usage : 53.31Kb
